In [1]:
from game2048 import Game
import sys

In [2]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
from lasagne.layers import DenseLayer
floatX = theano.config.floatX

Using gpu device 0: GeForce GTX 965M (CNMeM is disabled, CuDNN 4007)
/usr/local/lib/python3.4/dist-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
input_var = T.tensor4()
target_var = T.ivector()
_ = lasagne.layers.InputLayer(shape=(None, 4, 4, 20), input_var=input_var)
_ = DenseLayer(_, num_units=900, nonlinearity=lasagne.nonlinearities.rectify)
_ = DenseLayer(_, num_units=300, nonlinearity=lasagne.nonlinearities.rectify)
_ = DenseLayer(_, num_units=200, nonlinearity=lasagne.nonlinearities.rectify)
l_out = DenseLayer(_, num_units=4, nonlinearity=lasagne.nonlinearities.softmax)
prediction = lasagne.layers.get_output(l_out)
P = theano.function([input_var], prediction)

In [4]:
table ={2**(i+1):i for i in range(20)}
def make_input(grid, d=0):
    g0 = np.rot90(grid, -d)
    r = np.zeros(shape=(4,4,20), dtype=floatX)
    for i in range(4):
        for j in range(4):
            v = g0[i, j]
            if v:
                r[i, j][table[v]]=1
    return r

In [5]:
with np.load('model-0.npz') as f:
     param_values = [f['arr_%d' % i] for i in range(len(f.files))]
lasagne.layers.set_all_param_values(l_out, param_values)

In [8]:
N =100
s=0
slen=0
stat = {}
for i in range(N):
    game = Game()
    moves = [0,1,2,3]
    game_len = 0
    while not game.end:
        board = np.array([make_input(game.grid)], dtype=floatX)
        moves = np.argsort(P(board)[0])[::-1]
        for m in moves:
            if game.move(m):
                break
        game_len+=1
    s+= game.score
    slen+=game_len
    m = game.max()
    stat[m]=stat.get(m,0)+1
print(s/N, slen/N, stat)

7529.4 871.17 {1024: 63, 2048: 14, 128: 1, 512: 21, 256: 1}


In [9]:
N =1000
s=0
slen=0
stat = {}
for i in range(N):
    game = Game()
    moves = [0,1,2,3]
    game_len = 0
    while not game.end:
        board = np.array([make_input(game.grid)], dtype=floatX)
        moves = np.argsort(P(board)[0])[::-1]
        for m in moves:
            if game.move(m):
                break
        game_len+=1
    s+= game.score
    slen+=game_len
    m = game.max()
    stat[m]=stat.get(m,0)+1
print(s/N, slen/N, stat)

7459.652 862.87 {1024: 606, 2048: 145, 128: 3, 512: 214, 256: 32}
